In [5]:
import os
import random
import gym
import numpy as np
from collections import deque
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.optimizers import RMSprop
from keras.losses import MeanSquaredError

# Disable GPU to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

def OurModel(input_shape, action_space):
    X_input = Input(input_shape)
    X = Dense(250, activation="relu", kernel_initializer='he_uniform')(X_input)
    X = Dense(300, activation="relu", kernel_initializer='he_uniform')(X)
    X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X)
    X = Dense(action_space, activation="linear", kernel_initializer='he_uniform')(X)

    model = Model(inputs=X_input, outputs=X, name='CartPole_DQN_Model')
    model.compile(loss=MeanSquaredError(), optimizer=RMSprop(learning_rate=0.00025, rho=0.95, epsilon=0.01), metrics=["accuracy"])
    model.summary()
    return model

class DQNAgent:
    def __init__(self):
        self.env = gym.make('CartPole-v1', render_mode="human")
        self.state_size = self.env.observation_space.shape[0]
        self.action_size = self.env.action_space.n
        self.EPISODES = 170
        self.memory = deque(maxlen=2000)

        self.gamma = 0.95
        self.epsilon = 0.8
        self.epsilon_min = 0.001
        self.epsilon_decay = 0.999
        self.batch_size = 64
        self.train_start = 1000
        self.highest_reward = 0  # Track highest reward

        self.model = OurModel(input_shape=(self.state_size,), action_space=self.action_size)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        if len(self.memory) > self.train_start and self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def act(self, state):
        if np.random.random() <= self.epsilon:
            return random.randrange(self.action_size)
        return np.argmax(self.model.predict(state, verbose=0))

    def replay(self):
        if len(self.memory) < self.train_start:
            return
        minibatch = random.sample(self.memory, min(len(self.memory), self.batch_size))

        state = np.zeros((self.batch_size, self.state_size))
        next_state = np.zeros((self.batch_size, self.state_size))
        action, reward, done = [], [], []

        for i in range(self.batch_size):
            state[i] = minibatch[i][0]
            action.append(minibatch[i][1])
            reward.append(minibatch[i][2])
            next_state[i] = minibatch[i][3]
            done.append(minibatch[i][4])

        target = self.model.predict(state, verbose=0)
        target_next = self.model.predict(next_state, verbose=0)

        for i in range(self.batch_size):
            if done[i]:
                target[i][action[i]] = reward[i]
            else:
                target[i][action[i]] = reward[i] + self.gamma * np.amax(target_next[i])

        self.model.fit(state, target, batch_size=self.batch_size, verbose=0)

    def load(self, name):
        self.model = load_model(name, custom_objects={"mse": MeanSquaredError()})

    def save(self, name):
        self.model.save(name)

    def run(self):
        for e in range(self.EPISODES):
            reset_output = self.env.reset()
            state = reset_output if isinstance(reset_output, np.ndarray) else reset_output[0]
            state = np.reshape(state, [1, self.state_size])

            done = False
            total_reward = 0
            i = 0

            while not done:
                self.env.render()
                action = self.act(state)
                next_state, reward, done, _, _ = self.env.step(action)
                next_state = np.reshape(next_state, [1, self.state_size])

                total_reward += reward
                if done and i < self.env.spec.max_episode_steps - 1:
                    reward = -100  # Penalize early termination

                self.remember(state, action, reward, next_state, done)
                state = next_state
                i += 1

                if done:
                    print(f"Episode: {e}/{self.EPISODES}, Score: {i}, Reward: {total_reward}, Epsilon: {self.epsilon:.4f}")

                    # Auto-save only if new reward is greater than previous best
                    if total_reward > self.highest_reward:
                        self.highest_reward = total_reward
                        print(f"New highest reward: {self.highest_reward}, saving model...")
                        self.save("best-cartpole-dqn.h5")

                    if i == 500:
                        print("Saving trained model as cartpole-dqn.h5")
                        self.save("cartpole-dqn.h5")
                        return

                self.replay()

    def test(self):
        """Test the trained model on 10 episodes."""
        self.load("best-cartpole-dqn.h5")

        for e in range(10):  # Run 10 test episodes
            reset_output = self.env.reset()
            state = reset_output if isinstance(reset_output, np.ndarray) else reset_output[0]
            state = np.reshape(state, [1, self.state_size])

            done = False
            total_reward = 0
            i = 0

            while not done:
                self.env.render()
                action = np.argmax(self.model.predict(state, verbose=0))
                next_state, reward, done, _, _ = self.env.step(action)
                next_state = np.reshape(next_state, [1, self.state_size])

                total_reward += reward
                state = next_state
                i += 1

                if done:
                    print(f"Test Episode: {e}, Score: {i}, Total Reward: {total_reward}")
                    break

if __name__ == "__main__":
    agent = DQNAgent()
    agent.run()
    #agent.test()

Model: "CartPole_DQN_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 4)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 250)                 │           1,250 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 300)                 │          75,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 64)                  │          19,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 95,944 (374.78 KB)

 Trainable params: 95,944 (374.78 KB)

 Non-trainable params: 0 (0.00 B)

...

Episode: 68/170, Score: 295, Reward: 295.0, Epsilon: 0.1726
New highest reward: 295.0, saving model...
Episode: 69/170, Score: 204, Reward: 204.0, Epsilon: 0.1407
Episode: 70/170, Score: 251, Reward: 251.0, Epsilon: 0.1095
Episode: 71/170, Score: 119, Reward: 119.0, Epsilon: 0.0972
Episode: 72/170, Score: 65, Reward: 65.0, Epsilon: 0.0911
Episode: 73/170, Score: 181, Reward: 181.0, Epsilon: 0.0760


Episode: 74/170, Score: 349, Reward: 349.0, Epsilon: 0.0536
New highest reward: 349.0, saving model...
Episode: 75/170, Score: 259, Reward: 259.0, Epsilon: 0.0414
Episode: 76/170, Score: 328, Reward: 328.0, Epsilon: 0.0298
Episode: 77/170, Score: 236, Reward: 236.0, Epsilon: 0.0235
Episode: 78/170, Score: 273, Reward: 273.0, Epsilon: 0.0179
Episode: 79/170, Score: 263, Reward: 263.0, Epsilon: 0.0138
Episode: 80/170, Score: 261, Reward: 261.0, Epsilon: 0.0106
Episode: 81/170, Score: 11, Reward: 11.0, Epsilon: 0.0105
Episode: 82/170, Score: 10, Reward: 10.0, Epsilon: 0.0104
Episode: 83/170, Score: 10, Reward: 10.0, Epsilon: 0.0103
Episode: 84/170, Score: 10, Reward: 10.0, Epsilon: 0.0102
Episode: 85/170, Score: 11, Reward: 11.0, Epsilon: 0.0101
Episode: 86/170, Score: 10, Reward: 10.0, Epsilon: 0.0100
Episode: 87/170, Score: 10, Reward: 10.0, Epsilon: 0.0099
Episode: 88/170, Score: 9, Reward: 9.0, Epsilon: 0.0098
Episode: 89/170, Score: 9, Reward: 9.0, Epsilon: 0.0097
Episode: 90/170, Sc

Episode: 101/170, Score: 413, Reward: 413.0, Epsilon: 0.0041
New highest reward: 413.0, saving model...


Episode: 102/170, Score: 452, Reward: 452.0, Epsilon: 0.0026
New highest reward: 452.0, saving model...
Episode: 103/170, Score: 371, Reward: 371.0, Epsilon: 0.0018
Episode: 104/170, Score: 305, Reward: 305.0, Epsilon: 0.0013
Episode: 105/170, Score: 244, Reward: 244.0, Epsilon: 0.0010
Episode: 106/170, Score: 211, Reward: 211.0, Epsilon: 0.0010
Episode: 107/170, Score: 266, Reward: 266.0, Epsilon: 0.0010
Episode: 108/170, Score: 239, Reward: 239.0, Epsilon: 0.0010
Episode: 109/170, Score: 255, Reward: 255.0, Epsilon: 0.0010
Episode: 110/170, Score: 209, Reward: 209.0, Epsilon: 0.0010
Episode: 111/170, Score: 210, Reward: 210.0, Epsilon: 0.0010
Episode: 112/170, Score: 233, Reward: 233.0, Epsilon: 0.0010
Episode: 113/170, Score: 274, Reward: 274.0, Epsilon: 0.0010
Episode: 114/170, Score: 340, Reward: 340.0, Epsilon: 0.0010
Episode: 115/170, Score: 260, Reward: 260.0, Epsilon: 0.0010
Episode: 116/170, Score: 411, Reward: 411.0, Epsilon: 0.0010
Episode: 117/170, Score: 238, Reward: 238.

Episode: 122/170, Score: 476, Reward: 476.0, Epsilon: 0.0010
New highest reward: 476.0, saving model...
Episode: 123/170, Score: 257, Reward: 257.0, Epsilon: 0.0010
Episode: 124/170, Score: 264, Reward: 264.0, Epsilon: 0.0010
Episode: 125/170, Score: 227, Reward: 227.0, Epsilon: 0.0010
Episode: 126/170, Score: 249, Reward: 249.0, Epsilon: 0.0010
Episode: 127/170, Score: 281, Reward: 281.0, Epsilon: 0.0010
Episode: 128/170, Score: 266, Reward: 266.0, Epsilon: 0.0010
Episode: 129/170, Score: 343, Reward: 343.0, Epsilon: 0.0010
Episode: 130/170, Score: 301, Reward: 301.0, Epsilon: 0.0010
Episode: 131/170, Score: 220, Reward: 220.0, Epsilon: 0.0010
Episode: 132/170, Score: 249, Reward: 249.0, Epsilon: 0.0010
Episode: 133/170, Score: 246, Reward: 246.0, Epsilon: 0.0010
Episode: 134/170, Score: 212, Reward: 212.0, Epsilon: 0.0010
Episode: 135/170, Score: 345, Reward: 345.0, Epsilon: 0.0010
Episode: 136/170, Score: 181, Reward: 181.0, Epsilon: 0.0010
Episode: 137/170, Score: 209, Reward: 209.

In [7]:
import os
import random
import gym
import numpy as np
from collections import deque
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.optimizers import RMSprop
from keras.losses import MeanSquaredError

# Disable GPU to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

def OurModel(input_shape, action_space):
    X_input = Input(input_shape)
    X = Dense(500, activation="relu", kernel_initializer='he_uniform')(X_input)
    X = Dense(400, activation="relu", kernel_initializer='he_uniform')(X)
    X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X)
    X = Dense(action_space, activation="linear", kernel_initializer='he_uniform')(X)

    model = Model(inputs=X_input, outputs=X, name='CartPole_DQN_Model')
    model.compile(loss=MeanSquaredError(), optimizer=RMSprop(learning_rate=0.00025, rho=0.95, epsilon=0.01), metrics=["accuracy"])
    model.summary()
    return model

class DQNAgent:
    def __init__(self):
        self.env = gym.make('CartPole-v1', render_mode="human")
        self.state_size = self.env.observation_space.shape[0]
        self.action_size = self.env.action_space.n
        self.EPISODES = 170
        self.memory = deque(maxlen=2000)

        self.gamma = 0.95
        self.epsilon = 0.8
        self.epsilon_min = 0.001
        self.epsilon_decay = 0.999
        self.batch_size = 64
        self.train_start = 1000
        self.highest_reward = 0  # Track highest reward

        self.model = OurModel(input_shape=(self.state_size,), action_space=self.action_size)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        if len(self.memory) > self.train_start and self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def act(self, state):
        if np.random.random() <= self.epsilon:
            return random.randrange(self.action_size)
        return np.argmax(self.model.predict(state, verbose=0))

    def replay(self):
        if len(self.memory) < self.train_start:
            return
        minibatch = random.sample(self.memory, min(len(self.memory), self.batch_size))

        state = np.zeros((self.batch_size, self.state_size))
        next_state = np.zeros((self.batch_size, self.state_size))
        action, reward, done = [], [], []

        for i in range(self.batch_size):
            state[i] = minibatch[i][0]
            action.append(minibatch[i][1])
            reward.append(minibatch[i][2])
            next_state[i] = minibatch[i][3]
            done.append(minibatch[i][4])

        target = self.model.predict(state, verbose=0)
        target_next = self.model.predict(next_state, verbose=0)

        for i in range(self.batch_size):
            if done[i]:
                target[i][action[i]] = reward[i]
            else:
                target[i][action[i]] = reward[i] + self.gamma * np.amax(target_next[i])

        self.model.fit(state, target, batch_size=self.batch_size, verbose=0)

    def load(self, name):
        self.model = load_model(name, custom_objects={"mse": MeanSquaredError()})

    def save(self, name):
        self.model.save(name)

    def run(self):
        for e in range(self.EPISODES):
            reset_output = self.env.reset()
            state = reset_output if isinstance(reset_output, np.ndarray) else reset_output[0]
            state = np.reshape(state, [1, self.state_size])

            done = False
            total_reward = 0
            i = 0

            while not done:
                self.env.render()
                action = self.act(state)
                next_state, reward, done, _, _ = self.env.step(action)
                next_state = np.reshape(next_state, [1, self.state_size])

                total_reward += reward
                if done and i < self.env.spec.max_episode_steps - 1:
                    reward = -100  # Penalize early termination

                self.remember(state, action, reward, next_state, done)
                state = next_state
                i += 1

                if done:
                    print(f"Episode: {e}/{self.EPISODES}, Score: {i}, Reward: {total_reward}, Epsilon: {self.epsilon:.4f}")

                    # Auto-save only if new reward is greater than previous best
                    if total_reward > self.highest_reward:
                        self.highest_reward = total_reward
                        print(f"New highest reward: {self.highest_reward}, saving model...")
                        self.save("best-cartpole-dqn.h5")

                    if i == 500:
                        print("Saving trained model as cartpole-dqn.h5")
                        self.save("cartpole-dqn.h5")
                        return

                self.replay()

    def test(self):
        """Test the trained model on 10 episodes."""
        self.load("best-cartpole-dqn.h5")

        for e in range(10):  # Run 10 test episodes
            reset_output = self.env.reset()
            state = reset_output if isinstance(reset_output, np.ndarray) else reset_output[0]
            state = np.reshape(state, [1, self.state_size])

            done = False
            total_reward = 0
            i = 0

            while not done:
                self.env.render()
                action = np.argmax(self.model.predict(state, verbose=0))
                next_state, reward, done, _, _ = self.env.step(action)
                next_state = np.reshape(next_state, [1, self.state_size])

                total_reward += reward
                state = next_state
                i += 1

                if done:
                    print(f"Test Episode: {e}, Score: {i}, Total Reward: {total_reward}")
                    break

if __name__ == "__main__":
    agent = DQNAgent()
    #agent.run()
    agent.test()

Model: "CartPole_DQN_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)           │ (None, 4)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ (None, 500)                 │           2,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (None, 400)                 │         200,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_26 (Dense)                     │ (None, 64)                  │          25,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 228,694 (893.34 KB)

 Trainable params: 228,694 (893.34 KB)

 Non-trainable params: 0 (0.00 B)

Test Episode: 0, Score: 363, Total Reward: 363.0
Test Episode: 1, Score: 633, Total Reward: 633.0
Test Episode: 2, Score: 360, Total Reward: 360.0
Test Episode: 3, Score: 261, Total Reward: 261.0
Test Episode: 4, Score: 219, Total Reward: 219.0
Test Episode: 5, Score: 413, Total Reward: 413.0
Test Episode: 6, Score: 262, Total Reward: 262.0
Test Episode: 7, Score: 1002, Total Reward: 1002.0
Test Episode: 8, Score: 413, Total Reward: 413.0
Test Episode: 9, Score: 483, Total Reward: 483.0
